In [4]:
import open3d as o3d

pcd1 = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-21.ply")
pcd2 = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply")

# pcd1_down = pcd1.voxel_down_sample(voxel_size=0.05)
# pcd2_down = pcd2.voxel_down_sample(voxel_size=0.05)

merged_pcd = pcd1 + pcd2

o3d.io.write_point_cloud(".\Imgs\PLYs\Merge\merged_cloud.ply", merged_pcd)

[Open3D WARNING] Read PLY failed: unable to read file: .\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply


True

In [9]:
import numpy as np
import copy
import open3d as o3d


def demo_crop_geometry():
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry")
    print("5) Press 'S' to save the selected geometry")
    print("6) Press 'F' to switch to freeview mode")
    pcd_data = o3d.data.DemoICPPointClouds()
    pcd = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-21.ply")
    o3d.visualization.draw_geometries_with_editing([pcd])


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def prepare_data():
    # pcd_data = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-21.ply")
    target = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply")
    print("Visualization of two point clouds before manual alignment")
    draw_registration_result(source, target, np.identity(4))
    return source, target


def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()


def register_via_correspondences(source, target, source_points, target_points):
    corr = np.zeros((len(source_points), 2))
    corr[:, 0] = source_points
    corr[:, 1] = target_points
    # estimate rough transformation using correspondences
    print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))
    # point-to-point ICP for refinement
    print("Perform point-to-point ICP refinement")
    threshold = 0.03  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    draw_registration_result(source, target, reg_p2p.transformation)


def demo_manual_registration():
    print("Demo for manual ICP")
    source, target = prepare_data()

    # pick points from two point clouds and builds correspondences
    source_points = pick_points(source)
    target_points = pick_points(target)
    assert (len(source_points) >= 3 and len(target_points) >= 3)
    assert (len(source_points) == len(target_points))
    register_via_correspondences(source, target, source_points, target_points)
    print("")




In [10]:
demo_crop_geometry()
demo_manual_registration()

Demo for manual geometry cropping
1) Press 'Y' twice to align geometry with negative direction of y-axis
2) Press 'K' to lock screen and to switch to selection mode
3) Drag for rectangle selection,
   or use ctrl + left click for polygon selection
4) Press 'C' to get a selected geometry
5) Press 'S' to save the selected geometry
6) Press 'F' to switch to freeview mode
Demo for manual ICP
[Open3D WARNING] Read PLY failed: unable to read file: .\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #1773156 (-2.8, -1.5, -6.9) to add in queue.
[Open3D INFO] Picked point #1723876 (-0.28, -0.79, -3.7) to add in queue.
[Open3D INFO] Picked point #1718279 (0.019, -0.4, -1.9) to add in queue.


1) Please pick at le

In [ ]:
import cv2
import numpy as np

# Load images
imgL = cv2.imread('/path/to/left/image.png', 0)
imgR = cv2.imread('/path/to/right/image.png', 0)

# Camera matrices
cameraMatrixL = np.array([[1543.48, 0, 2015.13],
                          [0, 1541.46, 1031.24],
                          [0, 0, 1]])
cameraMatrixR = np.array([[1546.76, 0, 1850.32],
                          [0, 1545.62, 989.93],
                          [0, 0, 1]])

# Distortion coefficients, assuming no lens distortion for simplification
distCoeffsL = np.zeros(5)
distCoeffsR = np.zeros(5)

# Stereo rectify (adjust as per your calibration data)
R = np.eye(3)  # Adjust based on actual rotation data
T = np.array([152.854, -0.501341, 3.34597])  # Translation vector

# Rectify images
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(cameraMatrixL, distCoeffsL, cameraMatrixR, distCoeffsR, imgL.shape[::-1], R, T)
mapL1, mapL2 = cv2.initUndistortRectifyMap(cameraMatrixL, distCoeffsL, R1, P1, imgL.shape[::-1], cv2.CV_16SC2)
mapR1, mapR2 = cv2.initUndistortRectifyMap(cameraMatrixR, distCoeffsR, R2, P2, imgR.shape[::-1], cv2.CV_16SC2)
rectifiedL = cv2.remap(imgL, mapL1, mapL2, cv2.INTER_LINEAR)
rectifiedR = cv2.remap(imgR, mapR1, mapR2, cv2.INTER_LINEAR)

# Compute disparity
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(rectifiedL, rectifiedR)

# Re-project to 3D
points_3D = cv2.reprojectImageTo3D(disparity, Q)

# Filter points
mask = disparity > disparity.min()
out_points = points_3D[mask]

### Kitty Point cloudssssss

In [11]:
import open3d as o3d
import copy
import numpy as np

# Initialize functions
def draw_registration_result(source, target, transformation):
    """
    param: source - source point cloud
    param: target - target point cloud
    param: transformation - 4 X 4 homogeneous transformation matrix
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp], zoom=0.4459, front=[0.9288, -0.2951, -0.2242], lookat=[1.6784, 2.0612, 1.4451], up=[-0.3402, -0.9189, -0.1996])

def find_nearest_neighbors(source_pc, target_pc, nearest_neigh_num):
    # Find the closest neighbor for each anchor point through KDTree
    point_cloud_tree = o3d.geometry.KDTreeFlann(source_pc)
    # Find nearest target_point neighbor index
    points_arr = []
    for point in target_pc.points:
        [_, idx, _] = point_cloud_tree.search_knn_vector_3d(point, nearest_neigh_num)
        points_arr.append(source_pc.points[idx[0]])
    return np.asarray(points_arr)

In [15]:
def icp(source, target):
    source.paint_uniform_color([0.5, 0.5, 0.5])
    target.paint_uniform_color([0, 0, 1])
    #source_points = np.asarray(source.points) # source_points is len()=198835x3 <--> 198835 points that have (x,y,z) val
    target_points = np.asarray(target.points)
    # Since there are more source_points than there are target_points, we know there is not
    # a perfect one-to-one correspondence match. Sometimes, many points will match to one point,
    # and other times, some points may not match at all.

    transform_matrix = np.asarray([[0.862, 0.011, -0.507, 0.5], [-0.139, 0.967, -0.215, 0.7], [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
    source = source.transform(transform_matrix)

    # While loop variables
    curr_iteration = 0
    cost_change_threshold = 1
    curr_cost = 1000
    prev_cost = 10000

    while (True):
        # 1. Find nearest neighbors
        new_source_points = find_nearest_neighbors(source, target, 1)

        # 2. Find point cloud centroids and their repositions
        source_centroid = np.mean(new_source_points, axis=0)
        target_centroid = np.mean(target_points, axis=0)
        source_repos = np.zeros_like(new_source_points)
        target_repos = np.zeros_like(target_points)
        source_repos = np.asarray([new_source_points[ind] - source_centroid for ind in range(len(new_source_points))])
        target_repos = np.asarray([target_points[ind] - target_centroid for ind in range(len(target_points))])

        # 3. Find correspondence between source and target point clouds
        cov_mat = target_repos.transpose() @ source_repos

        U, X, Vt = np.linalg.svd(cov_mat)
        R = U @ Vt
        t = target_centroid - R @ source_centroid
        t = np.reshape(t, (1,3))
        curr_cost = np.linalg.norm(target_repos - (R @ source_repos.T).T)
        print("Curr_cost=", curr_cost)
        if ((prev_cost - curr_cost) > cost_change_threshold):
            prev_cost = curr_cost
            transform_matrix = np.hstack((R, t.T))
            transform_matrix = np.vstack((transform_matrix, np.array([0, 0, 0, 1])))
            # If cost_change is acceptable, update source with new transformation matrix
            source = source.transform(transform_matrix)
            curr_iteration += 1
        else:
            break
    print("\nIteration=", curr_iteration)
    # Visualize final iteration and print out final variables
    draw_registration_result(source, target, transform_matrix)
    return transform_matrix

In [13]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])
part_a = icp(source, target)

Curr_cost= 16.78920863750992
Curr_cost= 13.08584712859379
Curr_cost= 11.070860691975472
Curr_cost= 9.948424121114488
Curr_cost= 9.213518766447848
Curr_cost= 8.680943691549158
Curr_cost= 8.270811369877109
Curr_cost= 7.977670243451086
Curr_cost= 7.78563867638042
Curr_cost= 7.6604093757612945
Curr_cost= 7.578429760363084
Curr_cost= 7.524530114024888
Curr_cost= 7.489114455798301
Curr_cost= 7.465958777144513
Curr_cost= 7.451037736426044
Curr_cost= 7.441224203489374
Curr_cost= 7.434734105682555
Curr_cost= 7.430509492790676
Curr_cost= 7.427687389623323
Curr_cost= 7.425838479541511
Curr_cost= 7.424590424353081
Curr_cost= 7.423773127430918

Iteration= 21


In [16]:
source = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-21.ply")
target = o3d.io.read_point_cloud(".\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply")
print(source.points, target.points)
part_b = icp(source, target)

[Open3D WARNING] Read PLY failed: unable to read file: .\Imgs\PLYs\Merge\point_cloud_PLY_107761722_1200_13-09-2024-16-25-35.ply
std::vector<Eigen::Vector3d> with 2212577 elements.
Use numpy.asarray() to access data. std::vector<Eigen::Vector3d> with 2274959 elements.
Use numpy.asarray() to access data.
Curr_cost= 2004.253240304982
Curr_cost= 1542.4245166160804
Curr_cost= 1321.2048211162833
Curr_cost= 1021.2451844142612
Curr_cost= 868.4154877176161
Curr_cost= 821.0623051653275
Curr_cost= 802.9714491295267
Curr_cost= 790.4074308509959
Curr_cost= 780.0983054806212
Curr_cost= 770.5364059998877
Curr_cost= 762.4868079527321
Curr_cost= 756.7545561531053
Curr_cost= 752.6340511504112
Curr_cost= 749.3283819758389
Curr_cost= 746.4677803605504
Curr_cost= 743.933034921714
Curr_cost= 741.7142781391289
Curr_cost= 739.7521296135228
Curr_cost= 737.8888094881638
Curr_cost= 736.0400791503573
Curr_cost= 734.2643104706368
Curr_cost= 732.5239512472828
Curr_cost= 730.7268867039596
Curr_cost= 728.897843037742

In [17]:
import numpy as np
from sklearn.neighbors import NearestNeighbors


def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

ModuleNotFoundError: No module named 'sklearn'

In [3]:
import numpy as np
import cv2

def Reprojection3D(image, disparity, f, b):
    # Camera parameters
    cx = 1007.56  # principal point x from LEFT_CAM_FHD
    cy = 515.619  # principal point y from LEFT_CAM_FHD

    # Creating the Q matrix with updated values
    Q = np.array([
        [1, 0, 0, -cx],
        [0, 1, 0, -cy],
        [0, 0, 0, f],
        [0, 0, -1/b, 0]
    ])

    if disparity.dtype != np.float32:
        disparity = disparity.astype(np.float32)
    
    if image.dtype != np.float32:
        image = image.astype(np.float32)

    # Reprojecting to 3D
    points = cv2.reprojectImageTo3D(disparity, Q)
    mask = disparity > disparity.min()
    out_points = points[mask]
    out_colors = image[mask]
    verts = out_points.reshape(-1, 3)
    colors = out_colors.reshape(-1, 3).astype(np.uint8)
    verts = np.hstack([verts, colors])

    # Preparing to save in PLY format
    ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar blue
property uchar green
property uchar red
end_header
'''
    with open('zed_stereo.ply', 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')

# Assume f is the average of fx and fy from LEFT_CAM_FHD
f = (771.738 + 770.73) / 2
b = 152.854  # Baseline in mm

In [4]:
import cv2
disp = cv2.imread('.\Imgs\PLYs\Merge\DepthViewer_depthimage_107761722_1200_14-09-2024-12-27-53.png', cv2.IMREAD_GRAYSCALE )
imgg = cv2.imread('.\Imgs\PLYs\Merge\DepthViewer_Left_107761722_1200_14-09-2024-12-27-40.png')
Reprojection3D(imgg, disp, f, b)